In [83]:
import time

from django.utils import timezone

In [53]:
def add_to_home_timeline(account, status_data):
    status = Status(
        id=status_data["id"], created_at=status_data["created_at"], data=status_data
    )
    status.save()
    item = Timeline(
        account=account, status=status, tag=Timeline.Tag.HOME,
        server=account.server
    )
    item.save()

In [54]:
def create_toot_data(pk, content):
    created_at = timezone.now()
    toot_data = {
        "id": pk,
        "created_at": created_at,
        "content": content,
        "account": {
            "display_name": "someone",
        },
    }
    return toot_data


In [55]:
account = Account.objects.first()

In [60]:
add_to_home_timeline(account, create_toot_data(4, "blubber"))

In [16]:
toot = Status.objects.get(pk=456)

In [17]:
toot.content

In [18]:
toot.data

{'created_at': '2023-05-10T10:21:09.659Z',
 'id': 456,
 'data': {'id': 456,
  'created_at': '2023-05-10T10:21:09.659Z',
  'content': 'Hello World!',
  'account': {'display_name': 'someone'}}}

In [19]:
toot.data["content"]

KeyError: 'content'

In [52]:
Session.objects.first().get_decoded()

{'_auth_user_id': '1',
 '_auth_user_backend': 'django.contrib.auth.backends.ModelBackend',
 '_auth_user_hash': 'ae385ba0366ef53d8f533a801c8d2bc947150ac7e0dc8fe373fe0d1dd3bdc145',
 'seen_toots': {'1': True}}

In [57]:
Session.objects.first().save

<bound method Model.save of <Session: 094vrkscqmoy7nrr08qo7ufw8ywulgug>>

In [61]:
from getpass import getpass

from mastodon import Mastodon

In [62]:
def get_authenticated_client(api_base_url, username, password, client_name="pytooterapp"):
    client_id, secret = Mastodon.create_app(client_name, api_base_url=api_base_url)
    mastodon = Mastodon(
        client_id=client_id,
        client_secret=secret,
        api_base_url=api_base_url
    )
    print(mastodon)
    access_token = mastodon.log_in(username, password)
    mastodon.access_token = access_token
    return mastodon

In [63]:
username = "jochen-fediverse@wersdoerfer.de"
api_base_url = "https://fedi.wersdoerfer.de"
password = getpass()
mastodon = get_authenticated_client(api_base_url, username, password)

 ········


In [64]:
home_toots = mastodon.timeline_home()

In [65]:
len(home_toots)

20

In [66]:
toot = home_toots[0]

In [70]:
def add_toot_from_mastodon_to_home(account, toot):
    status = Status(pk=toot.id, created_at=toot.created_at, data=toot)
    status.save()
    item = Timeline(
        account=account, status=status, tag=Timeline.Tag.HOME,
        server=account.server
    )
    item.save()

In [80]:
add_toot_from_mastodon_to_home(account, home_toots[1])

In [85]:
while True:
    home_toots = mastodon.timeline_home()
    for toot in home_toots:
        add_toot_from_mastodon_to_home(account, toot)
    time.sleep(20)

MastodonNetworkError: Could not complete request: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [68]:
toot.created_at

datetime.datetime(2023, 5, 10, 10, 44, 21, tzinfo=tzlocal())

In [73]:
status = Status.objects.get(pk=110343976608713222)

In [79]:
home_toots[1].content

'<p>The downside of getting into gamedev and participating in a jam right now is that continuing work on what is hopefully going to be my submission is way more fun than debugging the Jekyll build of the OctoPrint plugin repository that I have to do right now. Alas, first work, then play.</p><p>Also, I really need to migrate the OctoPrint web pages to Hugo, that would save me many a headache...</p>'

In [74]:
status.display_name

'Aurin Azadî'

In [75]:
status.content

''